# Exposure Split Plots

Two separate exposure views sourced from `wolfpack/daily_snapshots.csv`:
- Plot 1: Gross vs Net exposure
- Plot 2: Long vs Short exposure


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO

from QuantConnect import *
from QuantConnect.Research import QuantBook

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

plt.style.use('seaborn-v0_8-whitegrid')

qb = QuantBook()
print('QuantBook initialized')


def read_csv_from_store(key):
    try:
        if not qb.ObjectStore.ContainsKey(key):
            print(f'ObjectStore key not found: {key}')
            return None
        content = qb.ObjectStore.Read(key)
        if not content:
            print(f'Empty ObjectStore key: {key}')
            return None
        return pd.read_csv(StringIO(content))
    except Exception as e:
        print(f'Error reading {key}: {e}')
        return None


In [ ]:
df = read_csv_from_store('wolfpack/daily_snapshots.csv')
if df is None:
    raise ValueError('wolfpack/daily_snapshots.csv is required')

required = ['date', 'gross_exposure', 'net_exposure', 'long_exposure', 'short_exposure']
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f'Missing required columns: {missing}')

df = df.copy()
df['date'] = pd.to_datetime(df['date'], errors='coerce')
for c in ['gross_exposure', 'net_exposure', 'long_exposure', 'short_exposure']:
    df[c] = pd.to_numeric(df[c], errors='coerce')

df = df.dropna(subset=['date']).sort_values('date').reset_index(drop=True)

print(f'Rows: {len(df):,}')
print(f'Date range: {df["date"].min().date()} to {df["date"].max().date()}')
df.tail(5)


In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
ax.plot(df['date'], 100 * df['gross_exposure'], label='gross_exposure', linewidth=2, color='#1f77b4')
ax.plot(df['date'], 100 * df['net_exposure'], label='net_exposure', linewidth=2, color='#ff7f0e')
ax.axhline(0, color='black', linewidth=1, alpha=0.4)
ax.set_title('Gross and Net Portfolio Exposure Over Time', fontsize=15, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Exposure (% of NAV)')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
ax.plot(df['date'], 100 * df['long_exposure'], label='long_exposure', linewidth=2, color='#2ca02c')
ax.plot(df['date'], 100 * df['short_exposure'], label='short_exposure', linewidth=2, color='#d62728')
ax.axhline(0, color='black', linewidth=1, alpha=0.4)
ax.set_title('Long and Short Portfolio Exposure Over Time', fontsize=15, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Exposure (% of NAV)')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
